In [1]:
import pandas as pd
import glob
from matplotlib import pyplot
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import warnings

from collections import Counter
from tqdm import tqdm
import os
import math

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# function that finds the indexes of non-anomalies for interpolation 
def interpolation_indexes(mylist, mynumber):
    
    left_neighbour = 0
    right_neighbour = 0
    
    # check left neighbour
    if((mynumber - 1) not in mylist):
        left_neighbour = mynumber - 1
    else:
        min_number = mynumber
        while min_number in mylist:
            min_number = min_number - 1
        left_neighbour = min_number
    
    # check right neighbour
    if((mynumber + 1) not in mylist):
        right_neighbour = mynumber + 1
    else:
        max_number = mynumber
        while max_number in mylist:
            max_number = max_number + 1
        right_neighbour = max_number
    
    return left_neighbour, right_neighbour

In [4]:
def lstm_model():
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
            kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X_train.shape[1])(L2)
    L4 = LSTM(8, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X_train.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='mae')
    return model

In [5]:
def train_anomaly_removal(df_train):
    
    # extract indexes for anomalies
    indexes = list(df_train[df_train.is_anomaly == 1].index)

    # creating a new df that replaces the anomalous samples with interpolation value
    df = pd.DataFrame(columns = df_train.columns)
    for i in range(0, len(df_train)):

        #print(i)

        # add all non-anomalies
        if(df_train.is_anomaly[i] == 0):
            df = df.append({'timestamp' : df_train.timestamp[i], 'value' : df_train.value[i], 'is_anomaly' : df_train.is_anomaly[i]},
            ignore_index = True)

        if(df_train.is_anomaly[i]==1):
            if(df_train.is_anomaly[i+1]!=1):
                #print(i)
                value_interpolation = (df_train.value[interpolation_indexes(indexes, i)[0]]+df_train.value[interpolation_indexes(indexes, i)[1]])/2

                df = df.append({'timestamp' : df_train.timestamp[i], 'value': value_interpolation, 'is_anomaly' : 0.0},
            ignore_index = True)
    return df

In [6]:
def threshold_computing_max(X_train):
    X_train_pred = model.predict(X_train, verbose=0)
    train_mae_loss_avg = np.mean(np.abs(X_train_pred - X_train), axis=1)
    max_threshold = np.max(train_mae_loss_avg)
    return max_threshold

In [7]:
def reconstruction_loss_predictions(X_test):
    X_test_pred = model.predict(X_test, verbose=0)
    mae_loss = np.mean(np.abs(X_test_pred-X_test), axis=1)
    return mae_loss, X_test_pred

In [8]:
def predicted_labels(mae_loss, threshold):
    predicted_test_label = []
    for i in range(0, len(test_mae_loss)):
        if(test_mae_loss[i][0]>(threshold)):
            predicted_test_label.append(1)
        else:
            predicted_test_label.append(0)
    return predicted_test_label

In [9]:
path_files_yahoo = '../datasets/Yahoo_A1Benchmark'

## Extract all file names corresponding to time series

In [10]:
ts_names = []
for i in os.listdir(path_files_yahoo):
    ts_names.append(str(i.split('.csv')[0]))

In [11]:
len(ts_names)

67

In [13]:
df_final_results_details = pd.DataFrame(columns = ['TS_name', 'lstmae_reconstruction_loss'])
df_final_results = pd.DataFrame(columns = ['TS_name', 'Labels_True', 'Labels_Pred', 'Test_Size', 'Model'])

scaler = MinMaxScaler()


for ts_name in tqdm(ts_names):
    
    print(ts_name)
    # path to train/test
    filename_yahoo = path_files_yahoo+ts_name+".csv"
    #print(filename_yahoo)
    
    # read ts
    df_yahoo = pd.read_csv(filename_yahoo)
    #print(len(df_yahoo))
    
    # split into train and test
    init_train = df_yahoo[0:math.floor(len(df_yahoo)/2)]
    #print(len(init_train))
    init_test = df_yahoo[math.floor(len(df_yahoo)/2):]
    #print(len(init_test))
    
    df_yahoo_train = init_train
    df_yahoo_test = init_test
    
    
    # remove anomalies from train to prepare LSTM
    # all anomalies are replaced by the interpolation of their closest non-anomalous neighbours
    df_train_yahoo = train_anomaly_removal(df_yahoo_train)
    
    
    # final training dataset + labels
    label_train = df_train_yahoo.is_anomaly
    train_yahoo = df_train_yahoo.value
    

    # final testing dataset + labels
    label_test = df_yahoo_test.is_anomaly
    test_yahoo = df_yahoo_test.value
    

    # Data preprocessing - Scaling
    # the scaler is fit on the training data and applied on the testing data
    train_yahoo_scale = scaler.fit_transform(np.array(train_yahoo).reshape(-1, 1))
    test_yahoo_scale = scaler.transform(np.array(test_yahoo).reshape(-1,1))
    

    # Shape Train Data for LSTM
    X_train = train_yahoo_scale.reshape(train_yahoo_scale.shape[0], 1, 1)

    # Train LSTM
    no_epochs = 50
    batch_size = 128
    model = lstm_model()
    encdec = model.fit(X_train, X_train, epochs=no_epochs, batch_size=batch_size,
                        validation_split=0.25).history

    # Threshold computing
    threshold = threshold_computing_max(X_train)
    
    # Shape Test Data for LSTM
    X_test = test_yahoo_scale.reshape(test_yahoo_scale.shape[0], 1, 1)
    
    test_mae_loss, X_test_pred = reconstruction_loss_predictions(X_test)
    
    # Extracting Predicted Labels
    predicted_test_label = predicted_labels(test_mae_loss, threshold)
    

    # Save Results
    # Save reconstruction Error for each Dataset
    df_results_details = pd.DataFrame()
    df_results_details['TS_name'] = [ts_name]
    df_results_details['lstmae_reconstruction_loss'] = threshold
    df_final_results_details = df_final_results_details.append(df_results_details)
    
    
    # Save Predicted Labels
    df_results = pd.DataFrame()
    
    
    df_results['TS_name'] = [ts_name]
    df_results['Labels_True'] = [list(label_test)]
    df_results['Labels_Pred'] = [predicted_test_label]
    df_results['Test_Size'] = len(list(label_test))
    df_results['Model'] = 'LSTM_AE'
    df_final_results = df_final_results.append(df_results)

  0%|          | 0/67 [00:00<?, ?it/s]

real_59
../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/Yahoo_A1Benchmark/real_59.csv
1423
711
712
Epoch 1/50
5/5 [==============================] - 4s 174ms/step - loss: 0.6678 - val_loss: 0.6157
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6606 - val_loss: 0.6091
Epoch 3/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6534 - val_loss: 0.6024
Epoch 4/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6460 - val_loss: 0.5955
Epoch 5/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6384 - val_loss: 0.5885
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6306 - val_loss: 0.5812
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 0.6226 - val_loss: 0.5736
Epoch 8/50
5/5 [==============================] - 0s 9ms/step - loss: 0.6143 - val_loss: 0.5658
Epoch 9/50
5/5 [==============================] - 0s 9ms/step - loss: 0.6056 - val_loss: 0.5577
Epoch 10/5

  1%|▏         | 1/67 [00:07<08:02,  7.31s/it]

real_65
../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/Yahoo_A1Benchmark/real_65.csv
1424
712
712
Epoch 1/50
5/5 [==============================] - 4s 130ms/step - loss: 0.4235 - val_loss: 0.5088
Epoch 2/50
5/5 [==============================] - 0s 9ms/step - loss: 0.4159 - val_loss: 0.5010
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 0.4080 - val_loss: 0.4929
Epoch 4/50
5/5 [==============================] - 0s 8ms/step - loss: 0.3999 - val_loss: 0.4846
Epoch 5/50
5/5 [==============================] - 0s 8ms/step - loss: 0.3916 - val_loss: 0.4760
Epoch 6/50
5/5 [==============================] - 0s 9ms/step - loss: 0.3831 - val_loss: 0.4672
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 0.3742 - val_loss: 0.4581
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 0.3652 - val_loss: 0.4486
Epoch 9/50
5/5 [==============================] - 0s 8ms/step - loss: 0.3559 - val_loss: 0.4390
Epoch 10/5

  3%|▎         | 2/67 [00:14<07:44,  7.15s/it]

...

 97%|█████████▋| 65/67 [06:06<00:10,  5.15s/it]

real_40
../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/Yahoo_A1Benchmark/real_40.csv
1427
713
714
Epoch 1/50
5/5 [==============================] - 3s 86ms/step - loss: 0.3429 - val_loss: 0.3125
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 0.3359 - val_loss: 0.3054
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 0.3286 - val_loss: 0.2977
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.3209 - val_loss: 0.2897
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 0.3129 - val_loss: 0.2815
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.3047 - val_loss: 0.2732
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.2963 - val_loss: 0.2645
Epoch 8/50
5/5 [==============================] - 0s 7ms/step - loss: 0.2876 - val_loss: 0.2556
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.2787 - val_loss: 0.2466
Epoch 10/50

 99%|█████████▊| 66/67 [06:12<00:05,  5.19s/it]

real_54
../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/Yahoo_A1Benchmark/real_54.csv
741
370
371
Epoch 1/50
3/3 [==============================] - 3s 169ms/step - loss: 0.4873 - val_loss: 0.5072
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4832 - val_loss: 0.5030
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4791 - val_loss: 0.4988
Epoch 4/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4750 - val_loss: 0.4945
Epoch 5/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4708 - val_loss: 0.4902
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 0.4665 - val_loss: 0.4859
Epoch 7/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4623 - val_loss: 0.4815
Epoch 8/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4580 - val_loss: 0.4772
Epoch 9/50
3/3 [==============================] - 0s 10ms/step - loss: 0.4536 - val_loss: 0.4729
Epoc

100%|██████████| 67/67 [06:16<00:00,  5.62s/it]


In [22]:
df_final_results = df_final_results.set_index([pd.Index(np.arange(len(df_final_results))), 'TS_name'])
df_final_results_details = df_final_results_details.set_index([pd.Index(np.arange(len(df_final_results_details))), 'TS_name'])



In [41]:
df_final_results_details.to_csv('./results/df_results_details_yahoo_lstmae_static.csv')

In [42]:
df_final_results.to_csv('./results/df_results_yahoo_lstmae_static.csv')